## Telegram to MT5 Bot

### Importing and installing libraries

In [1]:
pip install pyrogram

Note: you may need to restart the kernel to use updated packages.


In [2]:
import MetaTrader5 as mt5
from pyrogram import Client, filters
import re
from time import sleep

TgCrypto is missing! Pyrogram will work the same, but at a much slower speed. More info: https://docs.pyrogram.org/topics/speedups


### Defining function to pull data from telegram

In [3]:
#storing channel ID of telegram trade signal group
channel = {-1001416233252: {'type': 'channel', 'trading': 'str_long', 'url': 'test'}}

#List of forex sysmbols
symbols = ['AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'AUDUSD', 'CADCHF', 'CADJPY', 'CHFJPY', 'GBPAUD', 'GBPCAD',
           'GBPCHF', 'GBPJPY', 'GBPNZD', 'GBPUSD', 'EURAUD', 'EURCAD', 'EURCHF', 'EURGBP', 'EURJPY', 'EURNZD',
           'EURUSD', 'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDUSD', 'USDCAD', 'USDCHF', 'USDCNH', 'USDJPY', 'XAUUSD']

bot = Client("name", api_id="", api_hash="")

def sltp(chat_id, text, Sl, Tp):
    
    try:
        if chat_id == -1001416233252:  # test
            try:
                PRICE = float(re.findall(r'[\d.]+', str(text.split('\n')[0]))[0])
                SL = float(re.findall(r'[\d.]+', str([i for i in text.split('\n') if Sl in i]))[-1])
                TP = float(re.findall(r'[\d.]+', str([i for i in text.split('\n') if Tp in i][-1]))[-1])
                return [PRICE, SL, TP]
            except:
                return False
    except Exception as ex:
        bot.send_message(-1001247941772, f"sltp.{str(chat_id)}: {ex}")




In [4]:
def OrderSend(Symbol, Lot, Type, PRICE, Sl, Tp, Magic):
    selected = mt5.symbol_select(Symbol, True)
    if not selected:
        bot.send_message(-1001247941772, f"OrderSend.symbol_select: {str(mt5.last_error())}")
        mt5.shutdown()
    symbol_info = mt5.symbol_info(Symbol)
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": Symbol,
        "volume": Lot,
        "type": Type,
        "price": PRICE,
        "sl": Sl,
        "tp": Tp,
        "deviation": 3,
        "magic": Magic,
        "comment": "Order ochish",
        "type_time": mt5.ORDER_TIME_GTC,
        #"type_filling": mt5.ORDER_FILLING_    return,
    }
    result = mt5.order_send(request)
    mt5.shutdown()
    #quit()

### Managing telegram bot instance

In [5]:
@bot.on_message(filters.channel)
# @bot.on_message((filters.photo | filters.text) & (filters.channel | filters.chat))
def my_handler(client, message):
    Type = 2
    NOW_PRICE = 0
    Lot = 0.01
    chat_id = message.chat.id
    text = str(message.text).lower()
    if message.photo:
        if message.caption:
            text = message.caption
    if chat_id < 0:
        if 0 < len(text):
            if not ('limit' in text) and not ('sell stop' in text) and not ('buy stop' in text):
                if ('sl' in text and 'tp' in text) or ('stop loss' in text and 'take profit' in text):
                    for Symbol in symbols:
                        if Symbol.lower() in text:
                            print(Symbol)
                            if 'buy' in text:
                                Type = 0
                            if 'sell' in text:
                                Type = 1
                            st = sltp(chat_id, text, 'sl', 'tp')
                            print(st)
                            if st is not False and Type != 2:
                                for i in range(20):
                                    if mt5.initialize(login=68025724, server="RoboForex-DemoPro",
                                                      password="Metatrader5"):
                                        if abs(st[0] - NOW_PRICE) < 200 * mt5.symbol_info(Symbol).point:
                                            if Type == 0:
                                                NOW_PRICE = mt5.symbol_info_tick(Symbol).ask
                                            if Type == 1:
                                                NOW_PRICE = mt5.symbol_info_tick(Symbol).bid
                                            if mt5.symbol_info(Symbol) is not None:

                                                OrderSend(Symbol.upper(), Lot, Type, NOW_PRICE, st[1], st[2],
                                                          int(str(chat_id)[-10:]))
                                                break
                                            else:
                                                bot.send_message(-1001247941772,
                                                                 f"{str(mt5.last_error())}")
                                                OrderSend(Symbol.upper(), Lot, Type, NOW_PRICE, st[1], st[2],
                                                          int(str(chat_id)[-10:]))
                                                mt5.shutdown()
                                        else:
                                            mt5.shutdown()
                                            break
                                    sleep(5)


if __name__ == "__main__":
    bot.run()

C:\Users\Django\.conda\envs\mitoenv\lib\site-packages\pyrogram\methods\utilities\run.py:84: RuntimeWarning: coroutine 'Start.start' was never awaited
  self.start()


RuntimeError: This event loop is already running